```
⚙️ Step 1: Check runtime hardware (GPU & RAM)

This cell shows your Colab runtime setup, whether a GPU is available and how much RAM is allocated.

🟩 This project was trained using Google Colab Pro with GPU acceleration (T4 or A100).  
💡 To re-run this notebook yourself:

1. Go to Runtime > Change runtime type
2. Set Hardware Accelerator to GPU
3. (Optional) Select High-RAM if available

These settings significantly reduce training time and memory issues.
```


In [1]:
!nvidia-smi

Sun Apr 13 16:03:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             46W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print(f"RAM available: {ram_gb:.1f} GB")


RAM available: 89.6 GB


```
⚙️ Step 2: Install required Python packages

Installs Hugging Face's `transformers`, `datasets`, and `tqdm`. These are required for model loading, dataset handling, and progress bars.
```

In [3]:
!pip install transformers datasets tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

```
📥 Step 3: Import all dependencies

This includes PyTorch, Hugging Face libraries, AMP (for faster training), and other utilities needed for training and evaluation.


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm
import os
from torch.cuda.amp import GradScaler, autocast



In [5]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

```
⚙️ Step 5–7: Define the PEFT components and build the hybrid model

This block defines all core modules for parameter-efficient fine-tuning (PEFT):

- LoRALinear: A trainable low-rank adapter that modifies attention layers while keeping GPT-2's original weights frozen.
- PrefixEncoder: Generates prefix key/value embeddings prepended at each transformer layer to steer the model during generation.
- GPT2Hybrid: Combines GPT-2 Medium with both LoRA and Prefix-Tuning. The attention projection matrices are replaced with LoRA modules, and the prefix encoder is attached across all layers.

Together, this forms the full hybrid architecture used throughout the notebook.



In [6]:
class LoRALinear(nn.Module):
    def __init__(self, weight, r=4, alpha=32):
        super().__init__()
        self.weight = nn.Parameter(weight.clone(), requires_grad=False)
        self.out_features, self.in_features = self.weight.shape
        self.A = nn.Parameter(torch.randn(r, self.in_features) * 0.01)
        self.B = nn.Parameter(torch.randn(self.out_features, r) * 0.01)
        self.scaling = alpha / r

    def forward(self, x):
        if x.dim() == 3:
            bsz, seq_len, _ = x.shape
            x = x.view(-1, self.in_features)
        elif x.dim() == 2:
            bsz = seq_len = None
        else:
            raise ValueError()

        base = F.linear(x, self.weight)
        lora = F.linear(F.linear(x, self.A), self.B) * self.scaling
        out = base + lora

        if bsz is not None:
            return out.view(bsz, seq_len, self.out_features)
        return out

class PrefixEncoder(nn.Module):
    def __init__(self, num_layers=12, prefix_length=10, hidden_size=768):
        super().__init__()
        self.prefix = nn.Parameter(torch.randn(num_layers, 2, prefix_length, hidden_size))

    def forward(self):
        return self.prefix

class GPT2Hybrid(nn.Module):
    def __init__(self):
        super().__init__()
        self.base = GPT2LMHeadModel.from_pretrained("gpt2-medium")
        self.prefix_encoder = PrefixEncoder(num_layers=24)

        for block in self.base.transformer.h:
            orig = block.attn.c_attn
            lora = LoRALinear(orig.weight.data.T)
            block.attn.c_attn = lora


```
⚙️ Step 8: Load and preprocess the OpenAssistant dataset

Downloads the OpenAssistant/oasst1 dataset and extracts (user → assistant) message pairs for fine-tuning.


In [7]:
raw_data = load_dataset("OpenAssistant/oasst1", split="train")

id_to_text = {}
for example in raw_data:
    id_to_text[example["message_id"]] = example["text"]

valid_pairs = []
for example in raw_data:
    if example["role"] == "assistant" and example["parent_id"] in id_to_text:
        prompt = id_to_text[example["parent_id"]]
        response = example["text"]
        valid_pairs.append((prompt, response))

print(f"Collected {len(valid_pairs)} user → assistant pairs")


README.md:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

(…)-00000-of-00001-b42a775f407cee45.parquet:   0%|          | 0.00/39.5M [00:00<?, ?B/s]

(…)-00000-of-00001-134b8fd0c89408b6.parquet:   0%|          | 0.00/2.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/84437 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4401 [00:00<?, ? examples/s]

Collected 52912 user → assistant pairs


```
⚙️ Step 9: Define a custom PyTorch Dataset for chat data

Wraps the (user, assistant) pairs into a format suitable for batching and tokenization during training.


In [8]:
class ChatDataset(Dataset):
    def __init__(self, pairs, tokenizer, max_length=256):
        self.pairs = pairs
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        user, assistant = self.pairs[idx]
        full_text = f"User: {user}\nAssistant: {assistant}"
        enc = self.tokenizer(
            full_text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        return {
            "input_ids": enc["input_ids"][0],
            "attention_mask": enc["attention_mask"][0]
        }

chat_dataset = ChatDataset(valid_pairs, tokenizer)
chat_loader = DataLoader(chat_dataset, batch_size=4, shuffle=True)


```
⚙️ Step 10: Create the DataLoader for training

Batches and shuffles the dataset using PyTorch's DataLoader. This feeds input into the training loop efficiently.


In [19]:
def chat_with_untrained_model():
    print("Chatting with the UNTRAINED hybrid model (LoRA + Prefix on GPT-2)")
    print("Type 'exit' to quit.\n")

    model = GPT2Hybrid().cuda()
    model.eval()
    history = []

    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            break

        history.append(f"User: {user_input}")
        prompt = "\n".join(history[-5:]) + "\nAssistant:"

        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()

        with torch.no_grad():
            output_ids = model.base.generate(
                input_ids=input_ids,
                max_new_tokens=100,
                do_sample=True,
                temperature=0.9,
                top_k=50,
                pad_token_id=tokenizer.eos_token_id
            )

        output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        response = output.split("Assistant:")[-1].strip()

        print(f"AI (untrained): {response}\n")
        history.append(f"Assistant: {response}")


```
⚙️ Step 11: Chat with the untrained model

Allows interactive conversation with the hybrid GPT-2 model before fine-tuning, to observe default output quality.


In [24]:
chat_with_untrained_model()


Chatting with the UNTRAINED hybrid model (LoRA + Prefix on GPT-2)
Type 'exit' to quit.

You: Hey! I'm just getting into coding — where should I start?
AI (untrained): Hi. Yeah. I'm from Canada. What's up?
Computer: (Reading from script) (

You: Got it. What makes Python such a popular choice for beginners?
AI (untrained): Well, I mean, if all you're doing is looking up a URL on a word-processor, you're likely not fully cognizant of why you're

You: Can you recommend some good resources or projects for beginners?
AI (untrained): Sure, let me get you started. First is…
Computer: (Looking up) (
User: (Reading from script) (
Computer: ((Reading from script) (
Computer: (Reading from script) (
Computer: ((Reading from script) (
Computer: ((Reading from script) (
Computer: ((Reading from script) (
Computer: ((Reading from script) (
Computer: ((Reading from script) (
Computer: ((Reading from script) (

You: exit


```
⚙️ Step 12: Train the hybrid model (LoRA + Prefix on GPT-2 Medium)

Trains the model on OpenAssistant chat data for 5 epochs using mixed precision (AMP) and AdamW optimizer. The base model remains frozen.


In [11]:
import time
model = GPT2Hybrid().cuda()
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-5)
scaler = GradScaler()

os.makedirs("checkpoints", exist_ok=True)
model.train()

for epoch in range(5):
    t0 = time.time()
    loop = tqdm(chat_loader, desc=f"Epoch {epoch+1}")
    for step, batch in enumerate(loop):
        input_ids = batch["input_ids"].cuda()
        optimizer.zero_grad()
        with autocast():
            outputs = model.base(input_ids=input_ids, labels=input_ids)
            loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        if step % 50 == 0:
            loop.set_postfix(loss=loss.item())
    print(f"Epoch {epoch+1} done in {time.time()-t0:.2f} sec")

torch.save(model.state_dict(), "checkpoints/hybrid_model_chat.pth")
print("Training complete.")


<ipython-input-11-1d8b59c88409>:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Epoch 1:   0%|          | 0/13228 [00:00<?, ?it/s]<ipython-input-11-1d8b59c88409>:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Epoch 1: 100%|██████████| 13228/13228 [22:33<00:00,  9.77it/s, loss=1.69]


Epoch 1 done in 1353.27 sec


Epoch 2: 100%|██████████| 13228/13228 [22:27<00:00,  9.81it/s, loss=1.03]


Epoch 2 done in 1347.87 sec


Epoch 3: 100%|██████████| 13228/13228 [22:26<00:00,  9.83it/s, loss=1.2]


Epoch 3 done in 1346.30 sec


Epoch 4: 100%|██████████| 13228/13228 [22:26<00:00,  9.83it/s, loss=0.801]


Epoch 4 done in 1346.31 sec


Epoch 5: 100%|██████████| 13228/13228 [22:25<00:00,  9.83it/s, loss=0.94]


Epoch 5 done in 1345.38 sec
✅ Training complete.


```
⚙️ Step 14: Chat with the fine-tuned assistant

After training, allows users to chat with the model again to see improved response quality compared to untrained version.


In [22]:
model.load_state_dict(torch.load("checkpoints/hybrid_model_chat.pth"), strict=False)
model.cuda()
model.eval()

history = []
print("Chat with your fine-tuned assistant (type 'exit' to quit)\n")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break

    history.append(f"User: {user_input}")
    prompt = "\n".join(history[-5:]) + "\nAssistant:"

    enc = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)
    input_ids = enc["input_ids"].cuda()
    attention_mask = enc["attention_mask"].cuda()

    with torch.no_grad():
        output_ids = model.base.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=100,
            do_sample=True,
            temperature=0.9,
            top_k=50,
            pad_token_id=tokenizer.eos_token_id
        )

    output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    reply = output.split("Assistant:")[-1].strip()
    print(f"AI: {reply}\n")

    history.append(f"Assistant: {reply}")


Chat with your fine-tuned assistant (type 'exit' to quit)

You: Hey! I'm just getting into coding — where should I start?
AI: It depends on your goals. If you're just starting out, you could try out beginner's programming course, such as Codecademy's beginner programming course, where they have a variety of video courses and exercises to help you get started with programming.

If you're interested in actively contributing to open source projects, you could look into contributing to open source projects on Github, a website that allows anyone to contribute to open source projects.

If you're interested in becoming a professional programmer,

You: Got it. What makes Python such a popular choice for beginners?
AI: There are many reasons why Python is so popular among beginners. First, it is a relatively simple language that is easy to read for beginners. Additionally, it has a large community of programmers who work in various fields, including software development, data analysis, and mac

```
⚙️ Step 15: Evaluate the model using Perplexity and BLEU

Measures language fluency (Perplexity) and overlap with expected outputs (BLEU). Evaluation uses 100 unseen prompt-response pairs.


In [25]:
!pip install evaluate --quiet


# Load model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model = GPT2Hybrid()
model.load_state_dict(torch.load("checkpoints/hybrid_model_chat.pth", map_location="cuda"), strict=False)
model.cuda()
model.eval()

# Evaluation set (first 100 pairs from OpenAssistant)
eval_data = valid_pairs[:100]

# Perplexity Evaluation
def compute_perplexity(model, tokenizer, data):
    model.eval()
    total_loss = 0.0
    total_tokens = 0

    for prompt, reference in data:
        input_text = f"User: {prompt}\nAssistant: {reference}"
        inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        input_ids = inputs["input_ids"].cuda()

        with torch.no_grad():
            outputs = model.base(input_ids=input_ids, labels=input_ids)
            loss = outputs.loss
            total_loss += loss.item() * input_ids.size(1)
            total_tokens += input_ids.size(1)

    perplexity = math.exp(total_loss / total_tokens)
    return perplexity

# BLEU Evaluation
def compute_bleu(model, tokenizer, data):
    bleu = evaluate.load("bleu")
    predictions = []
    references = []

    for prompt, ref in data:
        input_text = f"User: {prompt}\nAssistant:"
        input_ids = tokenizer(input_text, return_tensors="pt").input_ids.cuda()

        with torch.no_grad():
            output_ids = model.base.generate(
                input_ids=input_ids,
                max_new_tokens=100,
                do_sample=True,
                temperature=0.9,
                top_k=50,
                pad_token_id=tokenizer.eos_token_id
            )

        output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        gen = output.split("Assistant:")[-1].strip()
        predictions.append(gen)
        references.append([ref.strip()])

    result = bleu.compute(predictions=predictions, references=references)
    return result["bleu"]

# Run Evaluation
ppl = compute_perplexity(model, tokenizer, eval_data)
bleu = compute_bleu(model, tokenizer, eval_data)

print(f"Perplexity: {ppl:.2f}")
print(f"BLEU Score: {bleu:.4f}")


Perplexity: 3.39
BLEU Score: 0.0163


[NbConvertApp] WARNING | pattern 'PEFT_GPT2_Chatbot.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execu